In [157]:
import pandas as pd

df = pd.read_json("msc_self_instruct.jsonl", lines=True)

In [158]:
answer_df = pd.DataFrame()
answer_df['dialog'] = None
answer_df['question'] = None
answer_df['gold_answer'] = None
answer_df['long_mem_result'] = None
answer_df['long_mem_answer'] = None
answer_df['long_mem_f1'] = None
answer_df['long_mem_rc'] = None
answer_df['long_mem_pre'] = None
answer_df['long_mem_recall_result'] = None
answer_df['long_mem_recall_answer'] = None
answer_df['long_mem_recall_f1'] = None
answer_df['long_mem_recall_rc'] = None
answer_df['long_mem_recall_pre'] = None

In [159]:
from datetime import datetime, timedelta

for i in range(500):
    # Add dialog
    current_time = datetime.strptime("2024/11/1 12:00", "%Y/%m/%d %H:%M")
    dialog_data = []
    for dialog_session in df['previous_dialogs'][i]:
        session = []
        for count in range(int(len(dialog_session['dialog'])/2)):
            chat_log = {"text":f"Allen:{dialog_session['dialog'][2*count]['text']}, Jack:{dialog_session['dialog'][2*count+1]['text']}", "time":current_time.strftime("%Y-%m-%dT%H:%M:%SZ")}
            session.append(chat_log)
            current_time = current_time + timedelta(minutes=1)
        dialog_data.append(session)
    answer_df.loc[i, 'dialog'] = str(dialog_data)
    
    # Add question & answer
    answer_df.loc[i, 'question'] = f"Jack:{df['self_instruct'][i]['B']}"
    answer_df.loc[i, 'gold_answer'] = f"{df['self_instruct'][i]['A']}"

In [160]:
answer_df.head()

,dialog,question,gold_answer,long_mem_result,long_mem_answer,long_mem_f1,long_mem_rc,long_mem_pre,long_mem_recall_result,long_mem_recall_answer,long_mem_recall_f1,long_mem_recall_rc,long_mem_recall_pre
0,"[[{'text': ""Allen:Hi! How are you doing tonigh...","Jack:Hey, remember that time we talked about m...",Taylor Swift!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[[{'text': 'Allen:Hello, how are you doing?, J...","Jack:Hey, remember that time we talked about o...",I eat a fresh and raw diet to save on groceries.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[[{'text': 'Allen:Hello what are doing today?,...","Jack:Hey, remember that time we talked about o...",I used to work in the human services field.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[[{'text': ""Allen:How are you? I'm tired of my...","Jack:Hey, remember that time we talked about o...",Burger King!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[[{'text': 'Allen:Hi, how are you doing today?...","Jack:Hey, remember that time we talked about o...",Three miles!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
answer_df.to_json("MSC_datasets.json", orient="records", lines=True)

In [161]:
import pandas as pd

answer_df = pd.read_json("MSC_datasets.json", lines=True)

Find result

In [1]:
import pandas as pd

answer_df = pd.read_json("MSC_eval.json", lines=True)

In [ ]:
from long_memory.component import WeaviateLongMemory

long_mem = WeaviateLongMemory()

In [104]:
other_instruct="You are Allen. Put useful information to evidence from the similar_snippets."

In [ ]:
error_list = []
for i in range(5):
    print(f'==={i}===')
    try:
        long_mem.del_memory()
        dialogs = []
        for dialog_session in eval(answer_df.loc[i, 'dialog']):
            dialogs.extend(dialog_session)
        long_mem.add_chat_logs(dialogs)
        question = f"{answer_df.loc[i, 'question']}"
        answer_df.loc[i, 'long_mem_result'] = str(long_mem.get_memory(question, recall=False))
        answer_df.loc[i, 'long_mem_recall_result'] = str(long_mem.get_memory(question, recall=True, other_instruct=other_instruct))
    except Exception as e:
        error_list.append(i)
        print(f'----error:{e}----')
    if (i+1)%20==0:
        answer_df.to_json("MSC_eval.json", orient="records", lines=True)
answer_df.to_json("MSC_eval.json", orient="records", lines=True)

In [137]:
error_list

[79]

Generate answer

In [106]:
answer_prompt = """Base on the following document and answer the question.
You are Allen and given a brief answer.

Question:{question}

Document:{docs}
"""

In [107]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def llm_create(prompt):
        messages = [{"role": "user", "content": prompt}]
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        return completion.choices[0].message.content

In [140]:
for i in range(100):
    print(f'==={i}===')
    p = answer_prompt.format(question=answer_df.loc[i, 'question'], docs=answer_df.loc[i, 'long_mem_result'])
    long_mem_answer = llm_create(p)
    answer_df.loc[i, 'long_mem_answer'] = long_mem_answer
    
    p = answer_prompt.format(question=answer_df.loc[i, 'question'], docs=answer_df.loc[i, 'long_mem_recall_result'])
    long_mem_recall_answer = llm_create(p)
    answer_df.loc[i, 'long_mem_recall_answer'] = long_mem_recall_answer
    
    if (i+1)%20==0:
        answer_df.to_json("MSC_eval.json", orient="records", lines=True)
answer_df.to_json("MSC_eval.json", orient="records", lines=True)

===0===
===1===
===2===
===3===
===4===
===5===
===6===
===7===
===8===
===9===
===10===
===11===
===12===
===13===
===14===
===15===
===16===
===17===
===18===
===19===
===20===
===21===
===22===
===23===
===24===
===25===
===26===
===27===
===28===
===29===
===30===
===31===
===32===
===33===
===34===
===35===
===36===
===37===
===38===
===39===
===40===
===41===
===42===
===43===
===44===
===45===
===46===
===47===
===48===
===49===
===50===
===51===
===52===
===53===
===54===
===55===
===56===
===57===
===58===
===59===
===60===
===61===
===62===
===63===
===64===
===65===
===66===
===67===
===68===
===69===
===70===
===71===
===72===
===73===
===74===
===75===
===76===
===77===
===78===
===79===
===80===
===81===
===82===
===83===
===84===
===85===
===86===
===87===
===88===
===89===
===90===
===91===
===92===
===93===
===94===
===95===
===96===
===97===
===98===
===99===


In [152]:
index=1
eval(answer_df['long_mem_recall_result'][index])

{'search times': 1,
 'used queries': ['Jack:Hey, remember that time we talked about our jobs and expenses? What was that one thing you said you did to save money?'],
 'searched memory': [{'text': "Jack expressed interest in learning how to save using coupons, and Allen's wife is willing to help. Jack also mentioned his demanding job and interest in studying economics.",
   'time': '2024/11/01 12:15'}],
 'thought': "I found relevant information about Jack's interest in saving money using coupons, which he discussed with me and my wife offered to help him.",
 'evidence': [{'text': "Jack expressed interest in learning how to save using coupons, and Allen's wife is willing to help.",
   'time': '2024/11/01 12:15'},
  {'text': 'Allen shares couponing success, winning $50 in coupons, while Jack faces challenges at work.',
   'time': '2024/11/01 12:07'}]}

In [153]:
answer_df['question'][index]

'Jack:Hey, remember that time we talked about our jobs and expenses? What was that one thing you said you did to save money?'

In [154]:
answer_df['gold_answer'][index]

'I eat a fresh and raw diet to save on groceries.'

In [131]:
eval(answer_df['long_mem_result'][index])

{'closest_summary': {'text': 'Allen is in prison for shoplifting and discusses personal experiences and tips for staying out of trouble with Jack, who loves his cats.',
  'time': '2024/11/01 12:00'},
 'similar_snippets': [{'text': 'Allen:I want to see my cat! At least my moms is visiting soon. Have you been arrested?, Jack:Arrested?!? You think so... Lol. No, they have not caught me yet.',
   'time': '2024/11/01 12:01'},
  {'text': 'Allen:I am in here for shoplifting, with a month to go. What have you got away with?, Jack:My dad taught me a lot. Even how to stay out of jail, fortunately! Lol..',
   'time': '2024/11/01 12:02'},
  {'text': "Allen:I wish I knew my dad. Do you have any tips for staying out of trouble?, Jack:Sure. Don't do drugs. Don't be stupid. Get married and obey your wife. Simple.",
   'time': '2024/11/01 12:03'},
  {'text': 'Allen:Sounds kind of boring! What do you do for fun?, Jack:Lol. I play with my cats. What could be funner?!?',
   'time': '2024/11/01 12:04'},
  

In [109]:
answer_df.head()

,dialog,question,gold_answer,long_mem_result,long_mem_answer,long_mem_f1,long_mem_rc,long_mem_pre,long_mem_recall_result,long_mem_recall_answer,long_mem_recall_f1,long_mem_recall_rc,long_mem_recall_pre
0,"[[{'text': ""Allen:Hi! How are you doing tonigh...","Jack:Hey, remember that time we talked about m...",Taylor Swift!,{'closest_summary': {'text': 'Allen and Jack d...,I mentioned I could get into Taylor Swift.,NaN,NaN,NaN,"{'search times': 1, 'used queries': ['Jack:Hey...",I mentioned I could get into Taylor Swift.,NaN,NaN,NaN
1,"[[{'text': 'Allen:Hello, how are you doing?, J...","Jack:Hey, remember that time we talked about o...",I eat a fresh and raw diet to save on groceries.,{'closest_summary': {'text': 'Allen talked abo...,I saved money on groceries by using couponing.,NaN,NaN,NaN,"{'search times': 1, 'used queries': ['Jack:Hey...",I saved money on groceries through couponing. ...,NaN,NaN,NaN
2,"[[{'text': 'Allen:Hello what are doing today?,...","Jack:Hey, remember that time we talked about o...",I used to work in the human services field.,{'closest_summary': {'text': 'Allen and Jack d...,I used to work in the human services field.,NaN,NaN,NaN,"{'search times': 1, 'used queries': ['Jack:Hey...",I used to work in the human services field.,NaN,NaN,NaN
3,"[[{'text': ""Allen:How are you? I'm tired of my...","Jack:Hey, remember that time we talked about o...",Burger King!,"{'closest_summary': {'text': ""Allen's part-tim...",I'm working at Burger King.,NaN,NaN,NaN,"{'search times': 1, 'used queries': [""Jack:Hey...",I'm working at Burger King.,NaN,NaN,NaN
4,"[[{'text': 'Allen:Hi, how are you doing today?...","Jack:Hey, remember that time we talked about o...",Three miles!,{'closest_summary': {'text': 'Allen and Jack d...,I mentioned that a three-mile walk is key for ...,NaN,NaN,NaN,"{'search times': 1, 'used queries': ['Jack:Hey...",I mentioned that a three-mile walk is key for ...,NaN,NaN,NaN


ROUGE-L Score

In [141]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

score_df = answer_df[:100]

def evaluate(col):
    def f1(row):
        scores = scorer.score(row['gold_answer'], row[f'{col}_answer'])
        return scores['rougeL'].fmeasure
    def rc(row):
        scores = scorer.score(row['gold_answer'], row[f'{col}_answer'])
        return scores['rougeL'].recall
    def pre(row):
        scores = scorer.score(row['gold_answer'], row[f'{col}_answer'])
        return scores['rougeL'].precision
    score_df[f'{col}_f1'] = score_df.apply(f1, axis=1)
    score_df[f'{col}_rc'] = score_df.apply(rc, axis=1)
    score_df[f'{col}_pre'] = score_df.apply(pre, axis=1)
evaluate('long_mem')
evaluate('long_mem_recall')

/tmp/ipykernel_1004512/2950608491.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df[f'{col}_f1'] = score_df.apply(f1, axis=1)
/tmp/ipykernel_1004512/2950608491.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df[f'{col}_rc'] = score_df.apply(rc, axis=1)
/tmp/ipykernel_1004512/2950608491.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [142]:
number = 100
long_mem_means = score_df[['long_mem_f1', 'long_mem_rc', 'long_mem_pre']][:number].mean()
long_mem_recall_means = score_df[['long_mem_recall_f1', 'long_mem_recall_rc', 'long_mem_recall_pre']][:number].mean()
print(long_mem_means)
print(long_mem_recall_means)

long_mem_f1     0.366521
long_mem_rc     0.671405
long_mem_pre    0.304553
dtype: float64
long_mem_recall_f1     0.368206
long_mem_recall_rc     0.743659
long_mem_recall_pre    0.287696
dtype: float64


In [144]:
score_df.to_json("MSC_score_v1.json", orient="records", lines=True)

In [151]:
score_df

,dialog,question,gold_answer,long_mem_result,long_mem_answer,long_mem_f1,long_mem_rc,long_mem_pre,long_mem_recall_result,long_mem_recall_answer,long_mem_recall_f1,long_mem_recall_rc,long_mem_recall_pre
0,"[[{'text': ""Allen:Hi! How are you doing tonigh...","Jack:Hey, remember that time we talked about m...",Taylor Swift!,{'closest_summary': {'text': 'Allen and Jack d...,I mentioned I could get into Taylor Swift.,0.400000,1.000000,0.250000,"{'search times': 1, 'used queries': ['Jack:Hey...",I mentioned I could get into Taylor Swift.,0.400000,1.000000,0.250000
1,"[[{'text': 'Allen:Hello, how are you doing?, J...","Jack:Hey, remember that time we talked about o...",I eat a fresh and raw diet to save on groceries.,"{'closest_summary': {'text': ""Jack expressed i...",I mentioned that I save money by using coupons...,0.171429,0.272727,0.125000,"{'search times': 1, 'used queries': ['Jack:Hey...","I mentioned that I save money using coupons, a...",0.181818,0.272727,0.136364
2,"[[{'text': 'Allen:Hello what are doing today?,...","Jack:Hey, remember that time we talked about o...",I used to work in the human services field.,{'closest_summary': {'text': 'Allen and Jack d...,I used to work in the human services field.,1.000000,1.000000,1.000000,"{'search times': 1, 'used queries': ['Jack:Hey...",I used to work in the human services field.,1.000000,1.000000,1.000000
3,"[[{'text': ""Allen:How are you? I'm tired of my...","Jack:Hey, remember that time we talked about o...",Burger King!,"{'closest_summary': {'text': ""Allen's part-tim...",Allen: I'm working at Burger King.,0.444444,1.000000,0.285714,"{'search times': 1, 'used queries': [""Jack:Hey...",I'm working at Burger King.,0.500000,1.000000,0.333333
4,"[[{'text': 'Allen:Hi, how are you doing today?...","Jack:Hey, remember that time we talked about o...",Three miles!,{'closest_summary': {'text': 'Allen and Jack d...,I mentioned that I liked to walk three miles f...,0.266667,1.000000,0.153846,"{'search times': 1, 'used queries': ['Jack:Hey...",I mentioned that I like to walk three miles fo...,0.266667,1.000000,0.153846
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,[[{'text': 'Allen:Good morning. My name is sop...,"Jack:Hey, remember that time we talked about l...",The alphabet!,{'closest_summary': {'text': 'Allen expresses ...,I was planning to learn the alphabet!,0.444444,1.000000,0.285714,"{'search times': 1, 'used queries': ['Jack:Hey...",I mentioned that I was planning to try to lear...,0.285714,1.000000,0.166667
96,"[[{'text': 'Allen:Hello, how are you tonight?,...","Jack:Hey, remember that time we talked about t...",Fish tacos with cilantro lime slaw.,"{'closest_summary': {'text': ""Allen loves chic...",Allen: My fiancé makes the BEST fish tacos!,0.285714,0.333333,0.250000,"{'search times': 1, 'used queries': ['Jack:Hey...",Allen: My fiancée makes the best fish tacos!,0.266667,0.333333,0.222222
97,"[[{'text': 'Allen:Hello, I finally got a chanc...","Jack:Hey, remember that time we talked about s...",Get married and obey your wife.,{'closest_summary': {'text': 'Allen discusses ...,The third piece of advice you gave me was prob...,0.000000,0.000000,0.000000,"{'search times': 2, 'used queries': ['Jack:Hey...","The third piece of advice Jack gave was to ""ge...",0.571429,1.000000,0.400000
98,"[[{'text': ""Allen:Hey, how are you? I was just...","Jack:Hey, remember that time we talked about y...",I'm paid entirely in broken watches.,"{'closest_summary': {'text': 'On November 1, 2...",I get paid entirely in broken watches.,0.857143,0.857143,0.857143,"{'search times': 1, 'used queries': ['Jack:Hey...","I get paid entirely in broken watches, which I...",0.631579,0.857143,0.500000
